# Семинар 2. Практическая часть №5
## Обработка текстов и анализ в стиле Orange (Google Colab)

**Цели работы:**
- осознать аналогию «аддонов» Orange и дополнительных библиотек Python;
- освоить базовую предварительную обработку текста (очистка, токенизация, стоп-слова, лемматизация);
- построить облако слов по корпусу текстов;
- выполнить кластеризацию текстов;
- обучить простую модель классификации текстов и оценить её качество.

## Блок 0. Установка и импорт библиотек (аналог аддонов)

В Orange мы подключаем аддоны через интерфейс.
В Google Colab аналогом будет установка дополнительных пакетов (`nltk`, `wordcloud`) и импорт нужных модулей.

In [ ]:
# Установка дополнительных библиотек (аналог установки аддонов в Orange)
!pip install nltk wordcloud -q

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud

from sklearn.datasets import fetch_20newsgroups, load_iris
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.manifold import MDS
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

In [ ]:
# Загрузка ресурсов NLTK для токенизации, стоп-слов и лемматизации
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

### Мини-пример: MDS для числовых данных

В качестве аналога нового функционала (как аддон с MDS в Orange)
возьмём числовой датасет Iris и понизим размерность методом `MDS`.

In [ ]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target
target_names_iris = iris.target_names

mds_num = MDS(n_components=2, random_state=42)
X_iris_2d = mds_num.fit_transform(X_iris)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_iris_2d[:, 0], X_iris_2d[:, 1], c=y_iris)
plt.title("MDS-проекция датасета Iris")
plt.xlabel("Компонента 1")
plt.ylabel("Компонента 2")
plt.legend(handles=scatter.legend_elements()[0],
           labels=target_names_iris,
           title="Вид")
plt.show()

## Блок 1. Загрузка корпуса текстов

В Orange используется встроенный корпус Grimm tales.
Здесь возьмём корпус новостных текстов `20 newsgroups` из `sklearn`.

In [ ]:
categories = ["sci.space", "rec.sport.baseball", "talk.politics.mideast"]

newsgroups = fetch_20newsgroups(
    subset="train",
    categories=categories,
    remove=("headers", "footers", "quotes"),
    shuffle=True,
    random_state=42,
)

texts = newsgroups.data
labels = newsgroups.target
target_names = newsgroups.target_names

print(f"Количество документов: {len(texts)}")
print("Категории:", target_names)
print("\nФрагмент первого документа:\n")
print(texts[0][:700])

## Блок 2. Предварительная обработка текста (№16)

Аналог виджета **Preprocess Text** в Orange: очистка, токенизация, стоп-слова, лемматизация.

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if len(t) > 2 and t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

raw_example = texts[0]
clean_example = preprocess_text(raw_example)

print("=== ОРИГИНАЛ ===")
print(raw_example[:500])
print("\n=== ПОСЛЕ ПРЕДОБРАБОТКИ ===")
print(clean_example[:500])

## Блок 3. Массовая предобработка и облако слов

Применяем функцию предобработки ко всем документам
и строим облако слов по всему корпусу.

In [ ]:
preprocessed_texts = [preprocess_text(t) for t in texts]

len(preprocessed_texts), preprocessed_texts[0][:200]

In [ ]:
all_text = " ".join(preprocessed_texts)

wordcloud = WordCloud(width=800, height=400).generate(all_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Облако слов по корпусу (после предобработки)")
plt.show()

## Блок 4. Кластеризация текстов (№17)

TF-IDF векторизация, косинусные расстояния, MDS-проекция и иерархическая кластеризация.

In [ ]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(preprocessed_texts)

X.shape

In [ ]:
dist_matrix = cosine_distances(X)
dist_matrix.shape

In [ ]:
mds = MDS(
    n_components=2,
    dissimilarity="precomputed",
    random_state=42,
    n_init=4,
    max_iter=300,
)
X_2d = mds.fit_transform(dist_matrix)

X_2d[:5]

In [ ]:
n_clusters = 3

cluster_model = AgglomerativeClustering(
    n_clusters=n_clusters,
    affinity="precomputed",
    linkage="average",
)
cluster_labels = cluster_model.fit_predict(dist_matrix)

np.bincount(cluster_labels)

In [ ]:
plt.figure(figsize=(8, 6))
for cluster_id in range(n_clusters):
    mask = cluster_labels == cluster_id
    plt.scatter(
        X_2d[mask, 0],
        X_2d[mask, 1],
        label=f"cluster {cluster_id}",
        alpha=0.6,
    )
plt.title("MDS-проекция TF-IDF и кластеры (Agglomerative)")
plt.xlabel("Компонента 1")
plt.ylabel("Компонента 2")
plt.legend()
plt.show()

In [ ]:
def print_examples_for_cluster(cluster_id: int, n_examples: int = 3):
    print(f"\n=== Кластер {cluster_id} ===")
    idxs = np.where(cluster_labels == cluster_id)[0][:n_examples]
    for idx in idxs:
        true_label = target_names[labels[idx]]
        print(f"\n--- Документ {idx}, истинная категория: {true_label}")
        print(texts[idx][:400], "...\n")

for cid in range(n_clusters):
    print_examples_for_cluster(cid, n_examples=2)

## Блок 5. Классификация текстов (№18)

Строим конвейер `TF-IDF + Logistic Regression`, обучаем и оцениваем качество.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels,
)

clf_pipeline = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                max_features=5000,
                stop_words="english",
                lowercase=True,
            ),
        ),
        ("clf", LogisticRegression(max_iter=1000)),
    ]
)

clf_pipeline.fit(X_train, y_train)
y_pred = clf_pipeline.predict(X_test)

print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(cm)

ax.set_xticks(range(len(target_names)))
ax.set_yticks(range(len(target_names)))
ax.set_xticklabels(target_names, rotation=45, ha="right")
ax.set_yticklabels(target_names)

ax.set_xlabel("Предсказанный класс")
ax.set_ylabel("Истинный класс")
ax.set_title("Матрица ошибок")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha="center", va="center", color="w")

plt.tight_layout()
plt.show()

In [ ]:
example_text = "NASA announced a new mission to Mars with a powerful new rocket."
pred_label = clf_pipeline.predict([example_text])[0]

print("Текст:", example_text)
print("Предсказанная категория:", target_names[pred_label])

## Блок 6. Выводы

В ходе практической работы:

- показан аналог «аддонов» Orange через установку и использование дополнительных библиотек в Google Colab;
- выполнена предобработка текстов (очистка, токенизация, стоп-слова, лемматизация);
- построено облако слов по корпусу документов;
- проведена кластеризация текстов на основе TF-IDF-признаков и MDS-проекции;
- обучена и оценена простая модель классификации текстов по тематикам.

Таким образом, задания Практической части №5 реализованы в среде Google Colab с сохранением логики оригинальных упражнений.